# Install dependencies

In [1]:
#!pip install -U git+https://github.com/huggingface/accelerate.git

In [2]:
# !pip install --upgrade comet_ml -qq

# Imports

In [1]:
import comet_ml

from accelerate import notebook_launcher
from accelerate.utils import set_seed

import gc
import os
import numpy as np
import pandas as pd

import torch
import torchaudio
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from transformers import Trainer, TrainingArguments
from tqdm import tqdm

from utils import clean_text
from whisper_dataset import WhisperDataset

[2023-09-13 14:18:03,425] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


# Constants

WHISPER_MODEL examples:

* 1) openai/whisper-small (suitable for testing functionality, not very accurate on sentences, but capable of recognizing individual words or phrases. Requires low computational resources)
* 2) openai/whisper-medium (recommended medium model)
* 3) openai/whisper-large (sufficiently accurate on large sentences, but requires significant computational resources)
* 4) openai/whisper-large-v2 (sufficiently accurate on large sentences, but requires significant computational resources)
* 5) lorenzoncina/whisper-medium-ru (a model finetuned on the Russian language - recommended for training on Russian)

In [2]:
os.environ["COMET_LOG_ASSETS"] = "True"

WHISPER_MODEL = 'openai/whisper-small'
DATASET_DIR = '/kaggle/input/it-spectrum-dataset/'

# Whisper initializing

In [3]:
processor = AutoProcessor.from_pretrained(WHISPER_MODEL)
model = AutoModelForSpeechSeq2Seq.from_pretrained(WHISPER_MODEL)

In [4]:
# setting the model's language and defining the task of transcription
model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="tatar", task="transcribe")

# Data initializing

Training dataset. When indexed, it returns a list containing:

* 1) filepath - path to the audio
* 2) text - transcribed text by annotators
* 3) input_features - audio features for prediction
* 4) labels - transcribed text by annotators converted into tokens
* 5) attention mask - an attention mask where each element indicates whether the model should pay attention to the token corresponding to the same index in the labels list.

In [5]:
class WhisperDataset(Dataset):
    def __init__(self, audio_dir: str, processor, max_length, only_char=True):
        self.audio_dir = audio_dir
        df = pd.read_csv(audio_dir[:-1] + '.csv', index_col='id')
        self.data = {}
        counter = 0
        for row in df.itertuples():
            if not os.path.exists(audio_dir + str(row[0]) + '.txt'):
                print('Отсутствует файл', str(row[0]) + '.txt')
                continue
            if not os.path.exists(audio_dir + str(row[0]) + '.wav'):
                print(f'Отсутствует файл', str(row[0]) + '.wav')
                continue
            self.data[counter] = {
                'text': str(row[0]) + '.txt',
                'audio': str(row[0]) + '.wav'
            }
            counter += 1
        self.len = counter - 1
        self.only_char = only_char
        del counter
        del df
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return self.len
    
    def _get_audio_sample_path(self, index):
        return self.audio_dir + self.data[index]['audio']
    
    def _get_audio_sample_label(self, index):
        label_path = self.audio_dir + self.data[index]['text']
        with open(label_path, 'r') as f:
            label = clean_text(f.read()) if self.only_char else f.read() # Не учитывает ошибки в заполнение .txt
        return label

    def __getitem__(self, idx):
        filepath = self._get_audio_sample_path(idx)
        text = self._get_audio_sample_label(idx)
        
        audio, sample_rate = torchaudio.load(filepath)
        audio = torch.reshape(audio, (-1,))
        
        tokenized = self.processor.tokenizer(
            text, return_tensors='pt', padding='max_length', return_attention_mask=True, 
            max_length=self.max_length
        )
        
        labels, attention_mask = tokenized['input_ids'][0], tokenized['attention_mask'][0]
        
        input_features = self.processor(audio, return_tensors="pt", sampling_rate=sample_rate).input_features[0]
        
        return {
            'input_features': input_features, 
            'labels': labels,
            'attention_mask': attention_mask
        }

In [7]:
# create train/val/test datasets
train_dataset_1 = WhisperDataset('../../tatar_asr_2/train/', processor, model.config.max_length)
train_dataset_2 = WhisperDataset('../../tatar_asr_1/train/', processor, model.config.max_length)
train_dataset = torch.utils.data.ConcatDataset([train_dataset_1, train_dataset_2])

valid_dataset = WhisperDataset('../../tatar_asr_2/valid/', processor, model.config.max_length)
test_dataset = WhisperDataset('../../tatar_asr_1/valid/', processor, model.config.max_length)

Отсутствует файл 331.26.txt
Отсутствует файл 338.17.txt
Отсутствует файл 177.2.txt
Отсутствует файл 18.3.txt
Отсутствует файл 83.3.txt
Отсутствует файл 339.14.txt
Отсутствует файл 103.3.txt
Отсутствует файл 334.12.txt
Отсутствует файл 327.4.txt
Отсутствует файл 477.26.txt
Отсутствует файл 329.3.txt
Отсутствует файл 328.11.txt
Отсутствует файл 337.23.txt
Отсутствует файл 68.3.txt
Отсутствует файл 121.3.txt
Отсутствует файл 305.6.txt
Отсутствует файл 436.4.txt
Отсутствует файл 95.1.txt
Отсутствует файл 303.27.txt
Отсутствует файл 315.4.txt
Отсутствует файл 340.3.txt
Отсутствует файл 302.1.txt
Отсутствует файл 335.8.txt
Отсутствует файл 138.2.txt
Отсутствует файл 153.2.txt
Отсутствует файл 99.3.txt
Отсутствует файл 196.3.txt
Отсутствует файл 314.2.txt
Отсутствует файл 333.3.txt
Отсутствует файл 310.12.txt
Отсутствует файл 120.2.txt
Отсутствует файл 8.3.txt
Отсутствует файл 87.2.txt
Отсутствует файл 119.3.txt
Отсутствует файл 103.2.txt
Отсутствует файл 119.2.txt
Отсутствует файл 496.19.txt

In [8]:
len(train_dataset), len(valid_dataset)

(173229, 10446)

## GPU runtime

In [9]:
torch.cuda.empty_cache()
gc.collect()

726

In [10]:
!nvidia-smi

Wed Sep 13 10:28:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 520.61.05    Driver Version: 520.61.05    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:02:00.0 Off |                    0 |
| N/A   

# Training

In [19]:
comet_ml.init( project_name = "TatAsr-whisper", experiment_name = "TatAsr-whisper-dataset-all")

In [23]:
def training_function():
    global model
    training_args = TrainingArguments(
        output_dir='./whisper-dataset-all', 
        overwrite_output_dir=True, 
        num_train_epochs=5,
        per_device_train_batch_size=7,
        save_steps=500, 
        save_total_limit=2,
        do_train=True,
    )
    
    set_seed(42)
    torch.manual_seed(7)
    
    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
    )
    trainer.train()

In [24]:
notebook_launcher(training_function, num_processes=2, mixed_precision='fp16')

Launching training on 2 GPUs.


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET WARNING: You are trying to log string value as a metric. This is not recommended.
COMET INFO: Experiment is live on comet.com https://www.comet.com/gumaonelove/huggingface/66d2f1af52004007b8917086107c326b

[W reducer.cpp:1300] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
[W reducer.cpp:1300] Warning: find_unused_parameters=True was specified in DDP constr

Step,Training Loss


KeyboardInterrupt: 

# Testing

In [11]:
def get_last_model(): 
    checkpoint_path = max(os.listdir('../whisper-dataset-all'), key=lambda x: int(x.split('-')[-1]) if 'checkpoint-' in x else 0)
    checkpoint_path = os.path.join('../whisper-dataset-all', checkpoint_path)
    return AutoModelForSpeechSeq2Seq.from_pretrained(checkpoint_path)

In [12]:
model = get_last_model()

## Test dataset test

In [17]:
def predict(model, dataset: WhisperDataset) -> pd.DataFrame:
    predicted_df = pd.DataFrame([], columns=['filename', 'pred', 'gt'])
    n = len(dataset)
    for idx in tqdm(range(100, 200)):
        item = dataset[idx]
        filepath = dataset._get_audio_sample_path(idx)
        text = dataset._get_audio_sample_label(idx)
        input_features = item['input_features']
        attention_mask = item['attention_mask']
        filename = filepath.replace('\\', '/').split('/')[-1]
        model = model.to('cuda')
    
        input_features = torch.stack([input_features]).to('cuda')
        generated_ids = model.generate(inputs=input_features, attention_mask=attention_mask)
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
        predicted_df.loc[len(predicted_df)] = [filename, transcription, text]
    return predicted_df

In [18]:
predicted_df = predict(model, valid_dataset)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:45<00:00,  2.18it/s]


In [19]:
predicted_df

,filename,pred,gt
0,785_394009940.wav,бинзонның ассистиянең сынар кадәр җылытылган к...,бензолны ацетиленны с кадәр җылытылган күмерле...
1,888_174014518.wav,икенчесе кысык күзле кып кызыл йөзендә зур бор...,икенчесе кысык күзле кып кызыл йөзендә зур бор...
2,67_721826785.wav,тавык күркә бытбылдык үрдәк каз итләре рөхсәт ...,тавык күркә бытбылдык үрдәк каз итләре рөхсәт ...
3,766_1304178405.wav,минем бик зур гүләтнең коллекциясе җыелды,минем бик зур бюллетень коллекциясе җыелды
4,886_443528311.wav,матбугат конференциясе дә атысы билгеле иде ин...,матбугат конференциясе датасы билгеле иде инде...
...,...,...,...
95,882_853742303.wav,хәзерге украина хакимияте берничә ел уйлаган м...,хәзерге украина хакимияте берничә ел уйлаган м...
96,838_1140452245.wav,бу бер өлеш турында зур булмаган инсидент була...,бу бәрелеш турында зур булмаган инцидент булар...
97,801_893117302.wav,әлбәттә безнең даими игътибар зонасында тышкы ...,әлбәттә безнең даими игътибар зонасында тышкы ...
98,880_987311976.wav,татарстан республикасы предприятиене рәхм оешм...,татарстан республикасы предприятиеләре һәм оеш...


In [20]:
# metric: accuracy
acc = sum((predicted_df['pred'] == predicted_df['gt'])) / len(predicted_df)
print(f'{acc * 100}%')

33.0%


## Custom sample test:

In [25]:
audio, sample_rate = librosa.load(os.path.join(DATASET_DIR, 'sample/eu.0124f456-13b8-4765-936a-36bfd483683e.wav'), sr=16000)

In [26]:
inputs = processor(audio, return_tensors='pt', sampling_rate=sample_rate)
input_features = inputs.input_features

In [27]:
generated_ids = model.generate(inputs=input_features)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (448) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [28]:
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(transcription)

Новая
